In [42]:
import pandas as pd
import re
import os
from scipy import spatial
from pprint import pprint
from datetime import date

from gensim import corpora
from gensim.models import LsiModel
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel
from gensim.matutils import corpus2dense

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import task2a_prep_functions as p

today = date.today()
d1 = today.strftime('%d%m%Y')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Henryy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Preprocessing

In [43]:
# read in dataframe from csv
data = pd.read_csv('results_scrapping.csv')
data.head()

,Unnamed: 0,Link,Place,Content
0,0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ..."
1,1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi..."
2,2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...
3,3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...
4,4,https://www.roughguides.com/usa/great-plains/,The Great Plains Travel Guide,The rolling hills and vast grasslands of the G...


In [44]:
# preprocess given text
def preprocess(data):
    # Remove punctuation
    data['content_processed'] = data['Content'].map(lambda x: re.sub('[,\.!?]', '', x))

    # Convert text to lowercase
    data['content_processed'] = data['content_processed'].map(lambda x: x.lower())

    stop_words = stopwords.words('english')
    stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'de', 'km', 'one', 'two'])

    def sent_to_words(sentences):
        for sentence in sentences:
            yield simple_preprocess(str(sentence), deacc=True) # -> True removes punctuations

    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

    # Tokenizing and remove punctuation of processed content
    data_list = data.content_processed.values.tolist()
    data_words = list(sent_to_words(data_list))

    # Lemmatization of processed content
    data_lemmatize = p.lemmatize_to_list([data_words])[0]

    # Remove stop words of processed content
    data_words = remove_stopwords(data_lemmatize)
    data['content_prep'] = data_words

    return data


data = preprocess(data)
data.head()

,Unnamed: 0,Link,Place,Content,content_processed,content_prep
0,0,https://www.roughguides.com/usa/hawaii/waikiki/,Waikiki,"Built on a reclaimed swamp, two miles east of ...",built on a reclaimed swamp two miles east of d...,"[build, reclaimed, swamp, mile, east, downtown..."
1,1,https://www.roughguides.com/usa/florida/florid...,The Florida Keys,"Folklore, films and widespread hearsay have gi...",folklore films and widespread hearsay have giv...,"[folklore, film, widespread, hearsay, give, fl..."
2,2,https://www.roughguides.com/usa/rockies/yellow...,Yellowstone National Park,America’s oldest and easily its most famous na...,america’s oldest and easily its most famous na...,"[america, old, easily, famous, national, park,..."
3,3,https://www.roughguides.com/usa/hawaii/big-isl...,The Big Island,Although the Big Island of Hawaii could hold a...,although the big island of hawaii could hold a...,"[although, big, island, hawaii, could, hold, i..."
4,4,https://www.roughguides.com/usa/great-plains/,The Great Plains Travel Guide,The rolling hills and vast grasslands of the G...,the rolling hills and vast grasslands of the g...,"[rolling, hill, vast, grassland, great, plain,..."


# Latent Semantic Analysis (LSA)
For LSA the following basic steps are required. These steps are combined in the Gensim LsiModel (yes for some reason they named it LSI) that will be used here.

## 1.  TF-IDF Vectorization
Goal is to create a document-term matrix that contains the tf-idf values for words within each document. A high tf-idf score represents a word that appears often in a document but not very often in the corpus. This means that this word is likely usefully for dokument classification. Words that appear often in a document but also often in the corpus will get a low tf-idf score.

## 2. Singular Value Decomposition (SVD) for dimensionality reduction
the resulting document-term matrix is a huge matrix with a lot of noisy and redundant information. Therefore, we want to reduce the dimensions to only a few latent topics that capture the relationships among the words and documents.

In [45]:
# create dictionary and corpus
corpus = data['content_prep']
dictionary = corpora.Dictionary(corpus)
print(dictionary)

Dictionary(14525 unique tokens: ['ala', 'along', 'always', 'apart', 'around']...)


In [46]:
# convert corpus to bag of words
bow = [dictionary.doc2bow(text) for text in corpus]
len(bow)

100

In [ ]:
# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=data['content_prep'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

In [ ]:
# build LSA model
lsa_model = LsiModel(bow, num_topics=10, id2word=dictionary)

In [ ]:
dt_matrix = corpus2dense(lsa_model[bow], len(lsa_model.projection.s)).T / lsa_model.projection.s
dt_matrix

## Inspect Topics
The matrix plots a score for each document for each topic.
Todo
- Find corresponding topics for each number
  - might be difficult since we don't even know if there is a word for each topic
  - maybe find words that define each topic from tf-idf matrix
- figure out how many topics we want

In [ ]:
pprint(lsa_model.print_topics())

## Predicting closest document to input based on document topic matrix

In [ ]:
# get place of the closest document for each word
# transform svd matrix to spacial KDtree
tree = spatial.KDTree(dt_matrix)

# transform a list of words with the fitted model to get their vector-representation
input = [['sun', 'beach'], ['city', 'town'], ['mountain', 'hiking']]
# Todo: Run Preprocessing over input


# transform words with dict to bow
input_bow = [dictionary.doc2bow(words) for words in input]

input_topics = lsa_model.__getitem__(input_bow)
# reformat to keep just values in tuples
input_vecs = []
for line in input_topics:
    input_vecs.append([y[1] for y in line])

# get closest document vector for each word vector
for i, input_vec in enumerate(input_vecs):
    query = tree.query(input_vec)
    print(f'"{input[i]}" > "{data.Place[query[1]]}" Distance: {query[0]}')

#### Script to train multiple LSA Models with different configurations and show differences in an Excel File

In [ ]:
# find the coherence score with a different number of topics
for i in range(2,11):

    model_name = f'lda_{d1}_' + f'{i}' + 'topics'
    dest_path = f'../results/{model_name}/'
    if not os.path.exists(dest_path):
        os.mkdir(f'../results/{model_name}/')

    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=data['content_prep'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    # save Coherence Score to lsa model results directory
    with open(f'../results/{model_name}/lsa_vis_prepared_' + str(i) + '_scores.txt', 'w') as f:
        f.write(f'Coherence Score: {coherence_score}\n')
        f.close()

    lsa_model = LsiModel(bow, num_topics=10, id2word=dictionary)

    dt_matrix = corpus2dense(lsa_model[bow], len(lsa_model.projection.s)).T / lsa_model.projection.s
    df_matrix = pd.DataFrame(dt_matrix)
    df_matrix.to_csv(f'../results/{model_name}/lda_vis_prepared_' + str(i) + '_matrix.csv', index=False, header=False)

    pprint(lsa_model.print_topics())

    # get place of the closest document for each word
    # transform svd matrix to spacial KDtree
    tree = spatial.KDTree(dt_matrix)

    # transform a list of words with the fitted model to get their vector-representation
    input = [['sun', 'beach'], ['city', 'town'], ['mountain', 'hiking']]
    # Todo: Run Preprocessing over input

    # transform words with dict to bow
    input_bow = [dictionary.doc2bow(words) for words in input]

    input_topics = lsa_model.__getitem__(input_bow)
    # reformat to keep just values in tuples
    input_vecs = []
    for line in input_topics:
        input_vecs.append([y[1] for y in line])

    # get closest document vector for each word vector
    with open(f'../results/{model_name}/lda_vis_prepared_' + str(i) + '_prediction.txt', 'w') as f:
        for i, input_vec in enumerate(input_vecs):
            query = tree.query(input_vec)
            f.write(f'{input[i]} > "{data.Place[query[1]]}" - Distance: {query[0]}\n')
        f.close()

[(0,
  '0.366*"city" + 0.167*"around" + 0.150*"town" + 0.144*"place" + 0.120*"la" + '
  '0.120*"mexico" + 0.118*"century" + 0.112*"day" + 0.111*"take" + '
  '0.110*"area"'),
 (1,
  '-0.448*"city" + -0.227*"mexico" + 0.221*"town" + 0.185*"beach" + '
  '0.129*"island" + 0.115*"village" + 0.115*"rio" + 0.104*"corsica" + '
  '0.104*"region" + 0.102*"forest"'),
 (2,
  '-0.329*"mexico" + 0.301*"barcelona" + 0.204*"art" + 0.200*"berlin" + '
  '0.172*"museum" + 0.161*"bar" + -0.123*"site" + -0.118*"mexican" + '
  '-0.115*"pyramid" + 0.108*"catalan"'),
 (3,
  '-0.533*"berlin" + 0.225*"madrid" + 0.210*"la" + -0.184*"wall" + '
  '0.178*"barcelona" + -0.129*"building" + 0.127*"plaza" + 0.126*"del" + '
  '-0.116*"east" + 0.107*"san"'),
 (4,
  '-0.302*"madrid" + -0.241*"rio" + 0.231*"barcelona" + -0.212*"berlin" + '
  '-0.176*"gay" + -0.174*"plaza" + 0.137*"art" + -0.118*"san" + 0.117*"town" + '
  '0.107*"mexico"'),
 (5,
  '-0.449*"rio" + 0.229*"madrid" + 0.157*"plaza" + -0.149*"school" + '
  '0.147